In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

%matplotlib inline

plt.style.use('seaborn-deep')
plt.rcParams['figure.figsize'] = (12,8)

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score

from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.pipeline import Pipeline

from itertools import chain

In [1]:
! git clone https://github.com/facebookresearch/fastText.git
% cd fastText
! pip install .

C:\Users\Natalia\Documents\kaggle_competition\fastText


Cloning into 'fastText'...


Processing c:\users\natalia\documents\kaggle_competition\fasttext
  Stored in directory: C:\Users\Natalia\AppData\Local\Temp\pip-ephem-wheel-cache-p7yt12_n\wheels\8f\81\d5\52fe297804eaf9e6f5247009402f2f4818465d388819faaa00
Successfully built fasttext


You are using pip version 19.0.2, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import fastText

In [5]:
% cd ..

C:\Users\Natalia\Documents\kaggle_competition


In [6]:
df_train = pd.read_csv('train.tsv', sep='\t', index_col=0)
df_test = pd.read_csv('test_nolabel.tsv', sep='\t', index_col=0)

In [23]:
from string import punctuation
from tqdm import tqdm
import re

In [13]:
!pip3 install pymorphy2
import pymorphy2

  Stored in directory: C:\Users\Natalia\AppData\Local\pip\Cache\wheels\9b\04\dd\7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt


You are using pip version 19.0.2, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [67]:
import nltk
# nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natalia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [85]:
def lemmatize(texts):
    stemmer = pymorphy2.MorphAnalyzer()
    lemmatized = []
    for text in tqdm(texts):
        text = [word for word in re.split('\W+', text)]
        tokens = [stemmer.parse(word.strip())[0].normal_form for word in text]
        tokens = list(filter(lambda x: x.strip() not in ['', ' ', '> <', '»', '«', '–'] and x not in punctuation \
                             and x not in stopwords, tokens))
        lemmatized.append(' '.join(tokens))
    return np.array(lemmatized, dtype='object')

In [86]:
#descriptions = np.concatenate((df_train.desc_text.values, df_test.desc_text.values), axis=None)
descriptions = df_train.desc_text.values
descriptions[:2]

array(['Продаю стол раскладной, деревянный, советский на века. Состояние осень хорошее. Торг уместен. Самовывоз ФМР',
       'Тарелки глубокие 6 шт. Блюдца, чашки по 6 шт. Все ранешний фарфор. В другом пакете современные блюдца, чашки и чайник с мишкой Тедди. Ничем практически не пользовались'],
      dtype=object)

In [87]:
lemmatized = lemmatize(descriptions)
lemmatized[:5]



  0%|          | 0/351281 [00:00<?, ?it/s]

  0%|          | 38/351281 [00:00<15:40, 373.55it/s]

  0%|          | 76/351281 [00:00<15:37, 374.65it/s]

  0%|          | 100/351281 [00:00<18:48, 311.06it/s]

  0%|          | 135/351281 [00:00<18:16, 320.27it/s]

  0%|          | 183/351281 [00:00<16:28, 355.21it/s]

  0%|          | 231/351281 [00:00<15:19, 381.84it/s]

  0%|          | 275/351281 [00:00<15:01, 389.46it/s]

  0%|          | 313/351281 [00:00<15:34, 375.45it/s]

  0%|          | 358/351281 [00:00<14:56, 391.25it/s]

  0%|          | 397/351281 [00:01<15:31, 376.50it/s]

  0%|          | 435/351281 [00:01<15:31, 376.72it/s]

  0%|          | 473/351281 [00:01<15:44, 371.37it/s]

  0%|          | 523/351281 [00:01<14:37, 399.80it/s]

  0%|          | 564/351281 [00:01<15:46, 370.43it/s]

  0%|          | 602/351281 [00:01<16:09, 361.84it/s]

  0%|          | 639/351281 [00:01<16:33, 353.09it/s]

  0%|          | 680/351281 [00:01<15:53, 367.70it/s]

  0%|          | 718/

  2%|▏         | 5964/351281 [00:16<16:12, 355.25it/s]

  2%|▏         | 6011/351281 [00:16<15:04, 381.71it/s]

  2%|▏         | 6050/351281 [00:16<16:01, 359.05it/s]

  2%|▏         | 6087/351281 [00:16<18:31, 310.64it/s]

  2%|▏         | 6125/351281 [00:16<17:32, 328.01it/s]

  2%|▏         | 6162/351281 [00:16<17:08, 335.67it/s]

  2%|▏         | 6207/351281 [00:16<15:50, 363.15it/s]

  2%|▏         | 6245/351281 [00:17<16:41, 344.36it/s]

  2%|▏         | 6281/351281 [00:17<16:39, 345.17it/s]

  2%|▏         | 6324/351281 [00:17<16:01, 358.89it/s]

  2%|▏         | 6362/351281 [00:17<15:47, 364.20it/s]

  2%|▏         | 6405/351281 [00:17<15:05, 380.99it/s]

  2%|▏         | 6444/351281 [00:17<17:03, 336.82it/s]

  2%|▏         | 6481/351281 [00:17<16:54, 339.73it/s]

  2%|▏         | 6525/351281 [00:17<15:47, 364.00it/s]

  2%|▏         | 6563/351281 [00:17<16:23, 350.61it/s]

  2%|▏         | 6604/351281 [00:18<15:42, 365.82it/s]

  2%|▏         | 6644/351281 [00:18<15:18, 375.1

  3%|▎         | 11835/351281 [00:32<16:52, 335.26it/s]

  3%|▎         | 11880/351281 [00:32<15:36, 362.39it/s]

  3%|▎         | 11921/351281 [00:32<15:05, 374.72it/s]

  3%|▎         | 11967/351281 [00:32<14:27, 391.00it/s]

  3%|▎         | 12008/351281 [00:32<15:09, 373.06it/s]

  3%|▎         | 12047/351281 [00:32<15:09, 372.86it/s]

  3%|▎         | 12085/351281 [00:33<15:27, 365.55it/s]

  3%|▎         | 12123/351281 [00:33<17:00, 332.30it/s]

  3%|▎         | 12158/351281 [00:33<17:48, 317.51it/s]

  3%|▎         | 12204/351281 [00:33<16:16, 347.09it/s]

  3%|▎         | 12249/351281 [00:33<15:18, 369.24it/s]

  3%|▎         | 12288/351281 [00:33<15:18, 369.14it/s]

  4%|▎         | 12332/351281 [00:33<14:42, 384.10it/s]

  4%|▎         | 12378/351281 [00:33<14:00, 403.33it/s]

  4%|▎         | 12420/351281 [00:33<15:21, 367.83it/s]

  4%|▎         | 12467/351281 [00:34<14:22, 392.77it/s]

  4%|▎         | 12521/351281 [00:34<13:24, 421.04it/s]

  4%|▎         | 12565/351281 [

  5%|▍         | 17446/351281 [00:48<14:41, 378.62it/s]

  5%|▍         | 17491/351281 [00:48<14:07, 393.66it/s]

  5%|▍         | 17533/351281 [00:48<13:53, 400.37it/s]

  5%|▌         | 17574/351281 [00:48<14:06, 394.24it/s]

  5%|▌         | 17614/351281 [00:48<16:19, 340.72it/s]

  5%|▌         | 17655/351281 [00:49<15:38, 355.44it/s]

  5%|▌         | 17693/351281 [00:49<15:22, 361.71it/s]

  5%|▌         | 17731/351281 [00:49<15:30, 358.48it/s]

  5%|▌         | 17772/351281 [00:49<15:05, 368.28it/s]

  5%|▌         | 17816/351281 [00:49<14:27, 384.45it/s]

  5%|▌         | 17856/351281 [00:49<15:11, 365.85it/s]

  5%|▌         | 17900/351281 [00:49<14:33, 381.60it/s]

  5%|▌         | 17944/351281 [00:49<14:00, 396.64it/s]

  5%|▌         | 17986/351281 [00:49<13:52, 400.23it/s]

  5%|▌         | 18029/351281 [00:49<13:37, 407.87it/s]

  5%|▌         | 18072/351281 [00:50<13:26, 413.39it/s]

  5%|▌         | 18114/351281 [00:50<14:20, 387.03it/s]

  5%|▌         | 18155/351281 [

  7%|▋         | 23255/351281 [01:04<16:50, 324.50it/s]

  7%|▋         | 23290/351281 [01:04<16:47, 325.53it/s]

  7%|▋         | 23325/351281 [01:04<16:29, 331.34it/s]

  7%|▋         | 23362/351281 [01:04<16:04, 339.96it/s]

  7%|▋         | 23403/351281 [01:04<15:18, 357.17it/s]

  7%|▋         | 23445/351281 [01:05<14:37, 373.71it/s]

  7%|▋         | 23487/351281 [01:05<14:26, 378.43it/s]

  7%|▋         | 23540/351281 [01:05<13:18, 410.37it/s]

  7%|▋         | 23583/351281 [01:05<13:22, 408.11it/s]

  7%|▋         | 23625/351281 [01:05<15:02, 362.94it/s]

  7%|▋         | 23668/351281 [01:05<14:40, 372.14it/s]

  7%|▋         | 23709/351281 [01:05<14:27, 377.75it/s]

  7%|▋         | 23748/351281 [01:05<14:25, 378.32it/s]

  7%|▋         | 23789/351281 [01:05<14:09, 385.41it/s]

  7%|▋         | 23828/351281 [01:06<14:18, 381.42it/s]

  7%|▋         | 23868/351281 [01:06<14:08, 385.99it/s]

  7%|▋         | 23916/351281 [01:06<13:25, 406.21it/s]

  7%|▋         | 23958/351281 [

...

 94%|█████████▍| 330917/351281 [15:20<00:57, 353.06it/s]

 94%|█████████▍| 330961/351281 [15:20<00:54, 371.77it/s]

 94%|█████████▍| 331003/351281 [15:20<00:53, 381.64it/s]

 94%|█████████▍| 331051/351281 [15:21<00:49, 405.37it/s]

 94%|█████████▍| 331093/351281 [15:21<00:53, 373.93it/s]

 94%|█████████▍| 331132/351281 [15:21<01:01, 325.05it/s]

 94%|█████████▍| 331170/351281 [15:21<00:59, 339.12it/s]

 94%|█████████▍| 331206/351281 [15:21<00:58, 344.40it/s]

 94%|█████████▍| 331253/351281 [15:21<00:53, 373.77it/s]

 94%|█████████▍| 331294/351281 [15:21<00:52, 382.09it/s]

 94%|█████████▍| 331334/351281 [15:21<00:52, 381.50it/s]

 94%|█████████▍| 331373/351281 [15:21<00:54, 367.00it/s]

 94%|█████████▍| 331411/351281 [15:22<00:53, 368.94it/s]

 94%|█████████▍| 331455/351281 [15:22<00:51, 385.96it/s]

 94%|█████████▍| 331495/351281 [15:22<00:56, 347.73it/s]

 94%|█████████▍| 331540/351281 [15:22<00:52, 372.95it/s]

 94%|█████████▍| 331583/351281 [15:22<00:51, 381.97it/s]

 94%|█████████

 96%|█████████▌| 336546/351281 [15:36<00:42, 342.77it/s]

 96%|█████████▌| 336587/351281 [15:36<00:40, 359.81it/s]

 96%|█████████▌| 336624/351281 [15:36<00:40, 359.39it/s]

 96%|█████████▌| 336663/351281 [15:36<00:39, 367.81it/s]

 96%|█████████▌| 336705/351281 [15:36<00:38, 380.77it/s]

 96%|█████████▌| 336744/351281 [15:36<00:38, 380.98it/s]

 96%|█████████▌| 336783/351281 [15:37<00:39, 370.84it/s]

 96%|█████████▌| 336826/351281 [15:37<00:37, 385.01it/s]

 96%|█████████▌| 336866/351281 [15:37<00:37, 387.43it/s]

 96%|█████████▌| 336905/351281 [15:37<00:38, 374.02it/s]

 96%|█████████▌| 336943/351281 [15:37<00:38, 372.78it/s]

 96%|█████████▌| 336981/351281 [15:37<00:39, 358.28it/s]

 96%|█████████▌| 337018/351281 [15:37<00:40, 348.73it/s]

 96%|█████████▌| 337054/351281 [15:37<00:40, 349.24it/s]

 96%|█████████▌| 337091/351281 [15:37<00:40, 352.45it/s]

 96%|█████████▌| 337127/351281 [15:38<00:40, 347.78it/s]

 96%|█████████▌| 337162/351281 [15:38<00:41, 336.68it/s]

 96%|█████████

 97%|█████████▋| 342358/351281 [15:51<00:23, 386.92it/s]

 97%|█████████▋| 342398/351281 [15:52<00:23, 374.62it/s]

 97%|█████████▋| 342446/351281 [15:52<00:22, 397.82it/s]

 97%|█████████▋| 342487/351281 [15:52<00:22, 396.47it/s]

 98%|█████████▊| 342528/351281 [15:52<00:22, 389.91it/s]

 98%|█████████▊| 342569/351281 [15:52<00:22, 385.44it/s]

 98%|█████████▊| 342612/351281 [15:52<00:21, 397.00it/s]

 98%|█████████▊| 342653/351281 [15:52<00:21, 394.21it/s]

 98%|█████████▊| 342693/351281 [15:52<00:23, 371.98it/s]

 98%|█████████▊| 342731/351281 [15:52<00:23, 368.13it/s]

 98%|█████████▊| 342773/351281 [15:52<00:22, 376.42it/s]

 98%|█████████▊| 342818/351281 [15:53<00:21, 395.08it/s]

 98%|█████████▊| 342858/351281 [15:53<00:21, 393.35it/s]

 98%|█████████▊| 342898/351281 [15:53<00:21, 382.07it/s]

 98%|█████████▊| 342946/351281 [15:53<00:20, 406.22it/s]

 98%|█████████▊| 342988/351281 [15:53<00:21, 384.70it/s]

 98%|█████████▊| 343028/351281 [15:53<00:21, 387.77it/s]

 98%|█████████

 99%|█████████▉| 348055/351281 [16:07<00:08, 379.34it/s]

 99%|█████████▉| 348095/351281 [16:07<00:08, 384.50it/s]

 99%|█████████▉| 348134/351281 [16:07<00:08, 361.77it/s]

 99%|█████████▉| 348171/351281 [16:07<00:09, 331.29it/s]

 99%|█████████▉| 348205/351281 [16:08<00:09, 324.57it/s]

 99%|█████████▉| 348239/351281 [16:08<00:09, 305.41it/s]

 99%|█████████▉| 348273/351281 [16:08<00:09, 313.52it/s]

 99%|█████████▉| 348308/351281 [16:08<00:09, 312.63it/s]

 99%|█████████▉| 348341/351281 [16:08<00:09, 312.48it/s]

 99%|█████████▉| 348377/351281 [16:08<00:08, 325.16it/s]

 99%|█████████▉| 348412/351281 [16:08<00:08, 332.01it/s]

 99%|█████████▉| 348460/351281 [16:08<00:07, 364.41it/s]

 99%|█████████▉| 348499/351281 [16:08<00:07, 369.90it/s]

 99%|█████████▉| 348541/351281 [16:08<00:07, 378.73it/s]

 99%|█████████▉| 348586/351281 [16:09<00:06, 392.71it/s]

 99%|█████████▉| 348626/351281 [16:09<00:07, 354.33it/s]

 99%|█████████▉| 348671/351281 [16:09<00:06, 376.83it/s]

 99%|█████████

array(['продавать стол раскладной деревянный советский век состояние осень хороший торг уместный самовывоз фмр',
       'тарелка глубокий 6 шт блюдце чашка 6 шт весь ранешний фарфор друг пакет современный блюдце чашка чайник мишка тедди ничто практически пользоваться',
       'новый этикетка размер l подойти размер маломерить размер померить ул 25 линия 74 любой удобный время',
       'складываться трость весь колесо вниз сплошной ручка стоить сложить вид капюшон закрывать бампер передний колесо одинарный снег каша ехать это аналог пегий перий си сплошной ручка безупречный состояние покупаться подготовка школа старший водить зимой ездить макларен перестать снег сдвоить колесо буксовать собирать компактно машина коляска сад школа ехать 2 неделя использовать 2 час',
       'неразлучник птичка 1 5 год продавать большой клетка кормушка поилка'],
      dtype=object)

In [96]:
np.savetxt('lemmatized_train_description.csv', lemmatized, delimiter=',', encoding='utf-8', fmt="%s")

In [95]:
descriptions_test = df_test.desc_text.values

lemmatized_desc_test = lemmatize(descriptions_test)
lemmatized_desc_test[:5]



  0%|          | 0/89251 [00:00<?, ?it/s]

  0%|          | 36/89251 [00:00<04:18, 345.40it/s]

  0%|          | 75/89251 [00:00<04:09, 356.95it/s]

  0%|          | 99/89251 [00:00<04:52, 304.65it/s]

  0%|          | 126/89251 [00:00<05:06, 290.77it/s]

  0%|          | 166/89251 [00:00<04:41, 316.17it/s]

  0%|          | 198/89251 [00:00<04:42, 315.21it/s]

  0%|          | 232/89251 [00:00<04:37, 321.13it/s]

  0%|          | 262/89251 [00:00<05:01, 295.28it/s]

  0%|          | 303/89251 [00:00<04:36, 321.79it/s]

  0%|          | 344/89251 [00:01<04:19, 342.50it/s]

  0%|          | 382/89251 [00:01<04:14, 349.33it/s]

  0%|          | 418/89251 [00:01<04:22, 337.88it/s]

  1%|          | 453/89251 [00:01<05:02, 293.66it/s]

  1%|          | 500/89251 [00:01<04:30, 328.32it/s]

  1%|          | 536/89251 [00:01<04:28, 330.98it/s]

  1%|          | 574/89251 [00:01<04:20, 340.85it/s]

  1%|          | 610/89251 [00:01<04:39, 317.39it/s]

  1%|          | 643/89251 [00:01<04:37,

  6%|▌         | 5504/89251 [00:16<04:01, 346.91it/s]

  6%|▌         | 5540/89251 [00:16<04:13, 329.84it/s]

  6%|▌         | 5578/89251 [00:17<04:12, 332.01it/s]

  6%|▋         | 5612/89251 [00:17<05:33, 251.08it/s]

  6%|▋         | 5641/89251 [00:17<05:26, 256.27it/s]

  6%|▋         | 5679/89251 [00:17<04:57, 281.04it/s]

  6%|▋         | 5710/89251 [00:17<04:51, 286.95it/s]

  6%|▋         | 5741/89251 [00:17<04:51, 286.01it/s]

  6%|▋         | 5771/89251 [00:17<05:16, 263.58it/s]

  6%|▋         | 5799/89251 [00:17<05:28, 254.27it/s]

  7%|▋         | 5840/89251 [00:18<04:53, 284.16it/s]

  7%|▋         | 5871/89251 [00:18<05:01, 276.86it/s]

  7%|▋         | 5905/89251 [00:18<04:44, 292.64it/s]

  7%|▋         | 5947/89251 [00:18<04:19, 320.65it/s]

  7%|▋         | 5988/89251 [00:18<04:03, 342.45it/s]

  7%|▋         | 6024/89251 [00:18<04:10, 332.43it/s]

  7%|▋         | 6061/89251 [00:18<04:04, 340.30it/s]

  7%|▋         | 6096/89251 [00:18<04:28, 309.76it/s]

  7%|▋    

 12%|█▏        | 10910/89251 [00:33<03:46, 345.82it/s]

 12%|█▏        | 10967/89251 [00:33<03:24, 382.07it/s]

 12%|█▏        | 11008/89251 [00:34<03:47, 344.25it/s]

 12%|█▏        | 11046/89251 [00:34<04:28, 291.13it/s]

 12%|█▏        | 11079/89251 [00:34<04:27, 292.40it/s]

 12%|█▏        | 11118/89251 [00:34<04:09, 313.30it/s]

 12%|█▏        | 11152/89251 [00:34<04:13, 308.05it/s]

 13%|█▎        | 11185/89251 [00:34<04:18, 302.48it/s]

 13%|█▎        | 11220/89251 [00:34<04:07, 315.12it/s]

 13%|█▎        | 11253/89251 [00:34<04:20, 298.89it/s]

 13%|█▎        | 11284/89251 [00:35<04:18, 301.93it/s]

 13%|█▎        | 11320/89251 [00:35<04:07, 315.42it/s]

 13%|█▎        | 11360/89251 [00:35<04:00, 323.95it/s]

 13%|█▎        | 11393/89251 [00:35<04:23, 295.43it/s]

 13%|█▎        | 11424/89251 [00:35<04:20, 299.02it/s]

 13%|█▎        | 11455/89251 [00:35<04:41, 276.63it/s]

 13%|█▎        | 11484/89251 [00:35<05:01, 257.60it/s]

 13%|█▎        | 11516/89251 [00:35<04:45, 272.4

 18%|█▊        | 16369/89251 [00:50<03:24, 356.13it/s]

 18%|█▊        | 16414/89251 [00:50<03:12, 378.25it/s]

 18%|█▊        | 16454/89251 [00:50<03:12, 378.29it/s]

 18%|█▊        | 16493/89251 [00:50<03:33, 340.59it/s]

 19%|█▊        | 16529/89251 [00:51<03:42, 327.14it/s]

 19%|█▊        | 16563/89251 [00:51<03:55, 308.24it/s]

 19%|█▊        | 16598/89251 [00:51<03:47, 319.05it/s]

 19%|█▊        | 16634/89251 [00:51<03:41, 327.86it/s]

 19%|█▊        | 16673/89251 [00:51<03:35, 336.56it/s]

 19%|█▊        | 16708/89251 [00:51<03:35, 335.85it/s]

 19%|█▉        | 16752/89251 [00:51<03:20, 360.87it/s]

 19%|█▉        | 16797/89251 [00:51<03:16, 368.87it/s]

 19%|█▉        | 16839/89251 [00:51<03:20, 360.50it/s]

 19%|█▉        | 16884/89251 [00:52<03:09, 382.66it/s]

 19%|█▉        | 16923/89251 [00:52<03:30, 343.52it/s]

 19%|█▉        | 16959/89251 [00:52<03:43, 323.29it/s]

 19%|█▉        | 16993/89251 [00:52<03:42, 324.63it/s]

 19%|█▉        | 17042/89251 [00:52<03:20, 360.2

 24%|██▍       | 21811/89251 [01:07<03:38, 308.89it/s]

 24%|██▍       | 21847/89251 [01:07<03:30, 320.28it/s]

 25%|██▍       | 21881/89251 [01:07<03:38, 308.48it/s]

 25%|██▍       | 21913/89251 [01:07<03:40, 305.40it/s]

 25%|██▍       | 21950/89251 [01:07<03:30, 319.18it/s]

 25%|██▍       | 21983/89251 [01:07<04:00, 279.30it/s]

 25%|██▍       | 22013/89251 [01:08<03:58, 282.21it/s]

 25%|██▍       | 22049/89251 [01:08<03:45, 298.23it/s]

 25%|██▍       | 22080/89251 [01:08<03:54, 286.86it/s]

 25%|██▍       | 22119/89251 [01:08<03:38, 306.64it/s]

 25%|██▍       | 22155/89251 [01:08<03:29, 320.29it/s]

 25%|██▍       | 22194/89251 [01:08<03:26, 325.15it/s]

 25%|██▍       | 22228/89251 [01:08<03:43, 299.33it/s]

 25%|██▍       | 22259/89251 [01:08<04:02, 276.07it/s]

 25%|██▍       | 22295/89251 [01:08<03:47, 294.84it/s]

 25%|██▌       | 22331/89251 [01:09<03:37, 307.21it/s]

 25%|██▌       | 22373/89251 [01:09<03:20, 333.94it/s]

 25%|██▌       | 22412/89251 [01:09<03:11, 348.7

...

 79%|███████▉  | 70530/89251 [03:36<01:09, 270.81it/s]

 79%|███████▉  | 70559/89251 [03:36<01:10, 265.89it/s]

 79%|███████▉  | 70603/89251 [03:36<01:02, 300.64it/s]

 79%|███████▉  | 70638/89251 [03:36<00:59, 312.46it/s]

 79%|███████▉  | 70678/89251 [03:36<00:56, 331.33it/s]

 79%|███████▉  | 70713/89251 [03:36<00:55, 336.48it/s]

 79%|███████▉  | 70748/89251 [03:37<00:56, 328.72it/s]

 79%|███████▉  | 70782/89251 [03:37<00:57, 323.30it/s]

 79%|███████▉  | 70826/89251 [03:37<00:52, 350.21it/s]

 79%|███████▉  | 70863/89251 [03:37<00:53, 344.30it/s]

 79%|███████▉  | 70899/89251 [03:37<00:54, 335.50it/s]

 79%|███████▉  | 70934/89251 [03:37<00:56, 325.79it/s]

 80%|███████▉  | 70968/89251 [03:37<01:00, 303.71it/s]

 80%|███████▉  | 71012/89251 [03:37<00:54, 332.00it/s]

 80%|███████▉  | 71047/89251 [03:38<00:58, 308.94it/s]

 80%|███████▉  | 71080/89251 [03:38<01:02, 291.86it/s]

 80%|███████▉  | 71115/89251 [03:38<00:59, 304.18it/s]

 80%|███████▉  | 71147/89251 [03:38<00:59, 306.3

 85%|████████▌ | 76068/89251 [03:53<00:40, 325.00it/s]

 85%|████████▌ | 76102/89251 [03:53<00:47, 278.01it/s]

 85%|████████▌ | 76132/89251 [03:53<00:47, 277.77it/s]

 85%|████████▌ | 76170/89251 [03:53<00:43, 301.27it/s]

 85%|████████▌ | 76202/89251 [03:53<00:43, 302.55it/s]

 85%|████████▌ | 76234/89251 [03:53<00:44, 295.08it/s]

 85%|████████▌ | 76269/89251 [03:53<00:42, 307.43it/s]

 86%|████████▌ | 76316/89251 [03:54<00:38, 333.04it/s]

 86%|████████▌ | 76353/89251 [03:54<00:37, 340.74it/s]

 86%|████████▌ | 76388/89251 [03:54<00:39, 327.38it/s]

 86%|████████▌ | 76437/89251 [03:54<00:35, 361.39it/s]

 86%|████████▌ | 76475/89251 [03:54<00:34, 366.53it/s]

 86%|████████▌ | 76513/89251 [03:54<00:36, 353.21it/s]

 86%|████████▌ | 76550/89251 [03:54<00:41, 304.90it/s]

 86%|████████▌ | 76590/89251 [03:54<00:39, 323.76it/s]

 86%|████████▌ | 76624/89251 [03:54<00:41, 307.38it/s]

 86%|████████▌ | 76657/89251 [03:55<00:41, 302.03it/s]

 86%|████████▌ | 76690/89251 [03:55<00:40, 306.7

 91%|█████████▏| 81569/89251 [04:09<00:23, 327.68it/s]

 91%|█████████▏| 81603/89251 [04:10<00:25, 300.82it/s]

 91%|█████████▏| 81639/89251 [04:10<00:24, 315.82it/s]

 92%|█████████▏| 81673/89251 [04:10<00:23, 322.03it/s]

 92%|█████████▏| 81706/89251 [04:10<00:24, 306.59it/s]

 92%|█████████▏| 81751/89251 [04:10<00:22, 336.93it/s]

 92%|█████████▏| 81790/89251 [04:10<00:21, 350.57it/s]

 92%|█████████▏| 81827/89251 [04:10<00:24, 306.92it/s]

 92%|█████████▏| 81860/89251 [04:10<00:25, 288.68it/s]

 92%|█████████▏| 81894/89251 [04:10<00:24, 302.18it/s]

 92%|█████████▏| 81926/89251 [04:11<00:25, 284.25it/s]

 92%|█████████▏| 81956/89251 [04:11<00:26, 274.02it/s]

 92%|█████████▏| 81991/89251 [04:11<00:24, 291.84it/s]

 92%|█████████▏| 82024/89251 [04:11<00:24, 299.27it/s]

 92%|█████████▏| 82062/89251 [04:11<00:22, 318.25it/s]

 92%|█████████▏| 82099/89251 [04:11<00:21, 327.15it/s]

 92%|█████████▏| 82133/89251 [04:11<00:21, 325.47it/s]

 92%|█████████▏| 82179/89251 [04:11<00:19, 356.5

 98%|█████████▊| 87112/89251 [04:26<00:07, 294.18it/s]

 98%|█████████▊| 87143/89251 [04:26<00:07, 297.27it/s]

 98%|█████████▊| 87179/89251 [04:26<00:06, 311.04it/s]

 98%|█████████▊| 87212/89251 [04:26<00:06, 313.58it/s]

 98%|█████████▊| 87245/89251 [04:27<00:06, 302.83it/s]

 98%|█████████▊| 87276/89251 [04:27<00:06, 299.89it/s]

 98%|█████████▊| 87307/89251 [04:27<00:06, 297.01it/s]

 98%|█████████▊| 87347/89251 [04:27<00:05, 320.53it/s]

 98%|█████████▊| 87385/89251 [04:27<00:05, 334.79it/s]

 98%|█████████▊| 87431/89251 [04:27<00:05, 363.92it/s]

 98%|█████████▊| 87469/89251 [04:27<00:04, 362.57it/s]

 98%|█████████▊| 87507/89251 [04:27<00:05, 326.32it/s]

 98%|█████████▊| 87545/89251 [04:27<00:05, 337.38it/s]

 98%|█████████▊| 87580/89251 [04:28<00:05, 308.08it/s]

 98%|█████████▊| 87629/89251 [04:28<00:04, 345.42it/s]

 98%|█████████▊| 87667/89251 [04:28<00:04, 353.41it/s]

 98%|█████████▊| 87704/89251 [04:28<00:04, 335.17it/s]

 98%|█████████▊| 87739/89251 [04:28<00:04, 325.5

array(['ботильон канадский 34 размер 35 маломерить материал замша шнуровка носиться весна состояние отличный',
       'зимний костюм куртка отличный состояние штаны иметься потёртость колено остальной дырка затяжка пр куртка подклад отстёгиваться тёплый зимний шапка подарок',
       'продать прекрасный стильный шустрый смартфон отличный состояние гарантия полный комплект использоваться чехол защитный стекло батарея держать отлично подарок новое защитный стекло защитный плёнка чехол характеристика android 7 0 поддержка 2 х sim карта процессор количество ядро 4 частота 1250 мгц память встроить 16 гб оперативный 1 гб экран ips 16 миллион цветок диагональ 5 5 1280x720 пикс стандарт сотовый связь работа 4g lte сеть камера разрешение 13 мпикс питание ёмкость аккумулятор ма число 3000 ма число причина продажа подарить',
       'горка купание присоска малыш купаться пара',
       'новый наличие размер 90 сантиметр доставка бердск бесплатно'],
      dtype=object)

In [326]:
lemmatized_desc_test.shape

(89251,)

In [97]:
np.savetxt('lemmatized_test_description.csv', descriptions_test, delimiter=',', encoding='utf-8', fmt="%s")

In [119]:
# df_train.loc[:, 'desc_text'] = lemmatized

# Apply fastText

In [292]:
def applyFastText(texts, target):
    y = target.apply(lambda x: '__label__' + str(x))

    fastText_df = pd.DataFrame(columns=['text', 'label'], index=np.arange(y.shape[0]), \
                           data=np.vstack([texts, y.values]).T)
    fastText_df.to_csv('to_train_ft.txt', index=False, sep=' ', encoding='utf-8')
    
    fastText_model = fastText.train_supervised('to_train_ft.txt', wordNgrams=2)
    
    return fastText_model    
    
def get_fastText_proba(model, X_train):
    y_proba = model.predict(list(X_train), k=2)
    probas_pos = []
    for i in np.arange(len(y_proba[0])):
        label = y_proba[0][i][0]
        if label == '__label__0':
            probas_pos.append(y_proba[1][i][1])
        else:
            probas_pos.append(y_proba[1][i][0])   
    return np.array(probas_pos)

In [293]:
fastText_model = applyFastText(lemmatized, df_train.sold_fast)

In [294]:
ft_probas_pos = get_fastText_proba(fastText_model, lemmatized)

In [296]:
np.array(ft_probas_pos)[:5]

array([0.27031934, 0.51763797, 0.10651568, 0.17362656, 0.08377719])

In [297]:
np.savetxt('TRAIN_fastText_desc_posproba.csv', np.array(ft_probas_pos), delimiter=',', encoding='utf-8')

In [186]:
# y_pred_ft = fastText_model.predict(list(fastText_df.text.values))[0]
# y_pred_ft = list(chain(*y_pred_ft))

In [187]:
# y_pred_ft = list(map(lambda x: x.replace('__label__', ''), y_pred_ft))
# true_labels = list(map(lambda x: x.replace('__label__', ''), fastText_df.label.values))

In [188]:
# print('Accuracy:', accuracy_score(true_labels, np.array(y_pred_ft)))
# print('Fscore:', f1_score(true_labels, np.array(y_pred_ft), average='weighted'))

Accuracy: 0.7889666677104654
Fscore: 0.7458280584999859


In [298]:
from sklearn.metrics import roc_auc_score

auc_val = roc_auc_score(df_train.sold_fast.values, ft_probas_pos)
auc_val

0.7669134248324669

#### Test values

In [299]:
test_ft_probas_pos = get_fastText_proba(fastText_model, lemmatized_desc_test)

In [300]:
np.savetxt('TEST_fastText_desc_posproba.csv', test_ft_probas_pos, delimiter=',', encoding='utf-8')

## Same deal, but for name

In [205]:
names = df_train.name_text.values
lemmatized_names = lemmatize(names)
np.savetxt('lemmatized_train_name.csv', lemmatized_names, delimiter=',', encoding='utf-8', fmt="%s")




  0%|          | 0/351281 [00:00<?, ?it/s]


  0%|          | 367/351281 [00:00<01:36, 3643.39it/s]


  0%|          | 724/351281 [00:00<01:37, 3613.02it/s]


  0%|          | 1067/351281 [00:00<01:38, 3548.03it/s]


  0%|          | 1428/351281 [00:00<01:38, 3558.69it/s]


  1%|          | 1792/351281 [00:00<01:37, 3574.99it/s]


  1%|          | 2142/351281 [00:00<01:38, 3544.27it/s]


  1%|          | 2504/351281 [00:00<01:37, 3558.97it/s]


  1%|          | 2831/351281 [00:00<01:41, 3448.10it/s]


  1%|          | 3188/351281 [00:00<01:40, 3476.35it/s]


  1%|          | 3550/351281 [00:01<01:39, 3510.76it/s]


  1%|          | 3906/351281 [00:01<01:39, 3507.38it/s]


  1%|          | 4250/351281 [00:01<01:40, 3458.23it/s]


  1%|▏         | 4592/351281 [00:01<01:41, 3428.76it/s]


  1%|▏         | 4949/351281 [00:01<01:40, 3462.57it/s]


  2%|▏         | 5296/351281 [00:01<01:40, 3457.23it/s]


  2%|▏         | 5649/351281 [00:01<01:39, 3471.25it/s]


  2%|▏         | 6012/3512

 14%|█▍        | 48674/351281 [00:14<01:28, 3407.18it/s]


 14%|█▍        | 49048/351281 [00:14<01:26, 3493.47it/s]


 14%|█▍        | 49400/351281 [00:14<01:27, 3442.65it/s]


 14%|█▍        | 49746/351281 [00:14<01:28, 3389.90it/s]


 14%|█▍        | 50095/351281 [00:14<01:28, 3411.99it/s]


 14%|█▍        | 50480/351281 [00:14<01:25, 3525.47it/s]


 14%|█▍        | 50835/351281 [00:14<01:25, 3493.96it/s]


 15%|█▍        | 51186/351281 [00:14<01:28, 3390.24it/s]


 15%|█▍        | 51545/351281 [00:14<01:27, 3440.55it/s]


 15%|█▍        | 51916/351281 [00:15<01:25, 3509.89it/s]


 15%|█▍        | 52269/351281 [00:15<01:26, 3467.00it/s]


 15%|█▍        | 52617/351281 [00:15<01:29, 3343.84it/s]


 15%|█▌        | 52954/351281 [00:15<01:29, 3314.83it/s]


 15%|█▌        | 53312/351281 [00:15<01:28, 3383.14it/s]


 15%|█▌        | 53652/351281 [00:15<01:28, 3380.80it/s]


 15%|█▌        | 54025/351281 [00:15<01:25, 3471.40it/s]


 15%|█▌        | 54380/351281 [00:15<01:25, 3476.87it/s]

 28%|██▊       | 97429/351281 [00:28<01:16, 3301.04it/s]


 28%|██▊       | 97775/351281 [00:28<01:15, 3340.09it/s]


 28%|██▊       | 98119/351281 [00:28<01:15, 3362.23it/s]


 28%|██▊       | 98510/351281 [00:28<01:12, 3502.85it/s]


 28%|██▊       | 98863/351281 [00:28<01:13, 3412.12it/s]


 28%|██▊       | 99207/351281 [00:28<01:14, 3402.90it/s]


 28%|██▊       | 99579/351281 [00:28<01:12, 3485.03it/s]


 28%|██▊       | 99929/351281 [00:29<01:12, 3481.90it/s]


 29%|██▊       | 100279/351281 [00:29<01:13, 3428.72it/s]


 29%|██▊       | 100628/351281 [00:29<01:12, 3439.43it/s]


 29%|██▊       | 100973/351281 [00:29<01:13, 3424.88it/s]


 29%|██▉       | 101336/351281 [00:29<01:11, 3476.63it/s]


 29%|██▉       | 101686/351281 [00:29<01:11, 3476.02it/s]


 29%|██▉       | 102034/351281 [00:29<01:11, 3469.62it/s]


 29%|██▉       | 102383/351281 [00:29<01:11, 3468.13it/s]


 29%|██▉       | 102737/351281 [00:29<01:11, 3481.86it/s]


 29%|██▉       | 103090/351281 [00:30<01:11, 348

 42%|████▏     | 145925/351281 [00:42<01:02, 3286.82it/s]


 42%|████▏     | 146256/351281 [00:42<01:02, 3276.84it/s]


 42%|████▏     | 146612/351281 [00:42<01:01, 3350.02it/s]


 42%|████▏     | 146971/351281 [00:42<00:59, 3411.45it/s]


 42%|████▏     | 147332/351281 [00:42<00:58, 3461.40it/s]


 42%|████▏     | 147680/351281 [00:42<00:59, 3449.15it/s]


 42%|████▏     | 148026/351281 [00:42<00:59, 3394.30it/s]


 42%|████▏     | 148391/351281 [00:43<00:58, 3459.97it/s]


 42%|████▏     | 148760/351281 [00:43<00:57, 3518.54it/s]


 42%|████▏     | 149133/351281 [00:43<00:56, 3571.90it/s]


 43%|████▎     | 149493/351281 [00:43<00:56, 3572.50it/s]


 43%|████▎     | 149851/351281 [00:43<00:57, 3494.03it/s]


 43%|████▎     | 150211/351281 [00:43<00:57, 3517.61it/s]


 43%|████▎     | 150564/351281 [00:43<00:57, 3513.63it/s]


 43%|████▎     | 150916/351281 [00:43<00:57, 3487.07it/s]


 43%|████▎     | 151266/351281 [00:43<00:58, 3412.19it/s]


 43%|████▎     | 151608/351281 [00:43<00

 55%|█████▌    | 193863/351281 [00:56<00:48, 3255.06it/s]


 55%|█████▌    | 194201/351281 [00:56<00:47, 3275.03it/s]


 55%|█████▌    | 194529/351281 [00:56<00:53, 2936.56it/s]


 55%|█████▌    | 194876/351281 [00:56<00:50, 3072.57it/s]


 56%|█████▌    | 195190/351281 [00:56<00:50, 3076.78it/s]


 56%|█████▌    | 195572/351281 [00:56<00:47, 3261.39it/s]


 56%|█████▌    | 195905/351281 [00:57<00:47, 3274.60it/s]


 56%|█████▌    | 196259/351281 [00:57<00:46, 3343.01it/s]


 56%|█████▌    | 196605/351281 [00:57<00:45, 3370.06it/s]


 56%|█████▌    | 196945/351281 [00:57<00:46, 3332.10it/s]


 56%|█████▌    | 197280/351281 [00:57<00:47, 3252.77it/s]


 56%|█████▋    | 197614/351281 [00:57<00:46, 3271.42it/s]


 56%|█████▋    | 197943/351281 [00:57<00:46, 3269.84it/s]


 56%|█████▋    | 198277/351281 [00:57<00:46, 3283.48it/s]


 57%|█████▋    | 198606/351281 [00:57<00:47, 3230.11it/s]


 57%|█████▋    | 198941/351281 [00:57<00:46, 3258.21it/s]


 57%|█████▋    | 199294/351281 [00:58<00

 69%|██████▊   | 241505/351281 [01:10<00:33, 3291.89it/s]


 69%|██████▉   | 241836/351281 [01:10<00:33, 3290.12it/s]


 69%|██████▉   | 242181/351281 [01:10<00:32, 3329.45it/s]


 69%|██████▉   | 242528/351281 [01:10<00:32, 3353.52it/s]


 69%|██████▉   | 242874/351281 [01:10<00:32, 3377.53it/s]


 69%|██████▉   | 243213/351281 [01:10<00:32, 3373.89it/s]


 69%|██████▉   | 243551/351281 [01:10<00:32, 3328.64it/s]


 69%|██████▉   | 243885/351281 [01:11<00:32, 3314.90it/s]


 70%|██████▉   | 244226/351281 [01:11<00:32, 3325.96it/s]


 70%|██████▉   | 244626/351281 [01:11<00:30, 3496.34it/s]


 70%|██████▉   | 244979/351281 [01:11<00:31, 3407.81it/s]


 70%|██████▉   | 245322/351281 [01:11<00:31, 3337.56it/s]


 70%|██████▉   | 245658/351281 [01:11<00:32, 3288.13it/s]


 70%|███████   | 245997/351281 [01:11<00:31, 3310.94it/s]


 70%|███████   | 246359/351281 [01:11<00:30, 3390.99it/s]


 70%|███████   | 246740/351281 [01:11<00:29, 3499.63it/s]


 70%|███████   | 247092/351281 [01:12<00

 82%|████████▏ | 289542/351281 [01:24<00:17, 3505.69it/s]


 83%|████████▎ | 289894/351281 [01:24<00:17, 3461.10it/s]


 83%|████████▎ | 290241/351281 [01:24<00:17, 3405.46it/s]


 83%|████████▎ | 290583/351281 [01:24<00:17, 3382.24it/s]


 83%|████████▎ | 290959/351281 [01:24<00:17, 3480.28it/s]


 83%|████████▎ | 291309/351281 [01:24<00:17, 3458.01it/s]


 83%|████████▎ | 291656/351281 [01:25<00:17, 3393.41it/s]


 83%|████████▎ | 292007/351281 [01:25<00:17, 3420.24it/s]


 83%|████████▎ | 292350/351281 [01:25<00:17, 3345.91it/s]


 83%|████████▎ | 292686/351281 [01:25<00:18, 3111.29it/s]


 83%|████████▎ | 293078/351281 [01:25<00:17, 3310.41it/s]


 84%|████████▎ | 293443/351281 [01:25<00:17, 3398.51it/s]


 84%|████████▎ | 293788/351281 [01:25<00:17, 3327.76it/s]


 84%|████████▎ | 294127/351281 [01:25<00:17, 3338.97it/s]


 84%|████████▍ | 294469/351281 [01:25<00:16, 3355.64it/s]


 84%|████████▍ | 294822/351281 [01:25<00:16, 3398.92it/s]


 84%|████████▍ | 295164/351281 [01:26<00

 96%|█████████▌| 337549/351281 [01:38<00:04, 3388.37it/s]


 96%|█████████▌| 337889/351281 [01:38<00:03, 3384.45it/s]


 96%|█████████▋| 338247/351281 [01:38<00:03, 3433.60it/s]


 96%|█████████▋| 338592/351281 [01:38<00:03, 3331.87it/s]


 96%|█████████▋| 338932/351281 [01:38<00:03, 3344.79it/s]


 97%|█████████▋| 339292/351281 [01:38<00:03, 3410.38it/s]


 97%|█████████▋| 339666/351281 [01:39<00:03, 3495.82it/s]


 97%|█████████▋| 340017/351281 [01:39<00:03, 3431.14it/s]


 97%|█████████▋| 340362/351281 [01:39<00:03, 3349.59it/s]


 97%|█████████▋| 340712/351281 [01:39<00:03, 3386.15it/s]


 97%|█████████▋| 341076/351281 [01:39<00:02, 3451.32it/s]


 97%|█████████▋| 341440/351281 [01:39<00:02, 3498.45it/s]


 97%|█████████▋| 341798/351281 [01:39<00:02, 3514.94it/s]


 97%|█████████▋| 342151/351281 [01:39<00:02, 3352.13it/s]


 97%|█████████▋| 342489/351281 [01:39<00:02, 3343.21it/s]


 98%|█████████▊| 342827/351281 [01:39<00:02, 3346.88it/s]


 98%|█████████▊| 343177/351281 [01:40<00

In [208]:
test_names = df_test.name_text.values
test_lemmatized_names = lemmatize(test_names)
np.savetxt('lemmatized_test_name.csv', test_lemmatized_names, delimiter=',', encoding='utf-8', fmt="%s")




  0%|          | 0/89251 [00:00<?, ?it/s]


  0%|          | 358/89251 [00:00<00:25, 3554.09it/s]


  1%|          | 725/89251 [00:00<00:24, 3580.41it/s]


  1%|          | 1056/89251 [00:00<00:25, 3486.69it/s]


  2%|▏         | 1355/89251 [00:00<00:26, 3313.11it/s]


  2%|▏         | 1691/89251 [00:00<00:26, 3319.84it/s]


  2%|▏         | 2060/89251 [00:00<00:25, 3415.90it/s]


  3%|▎         | 2385/89251 [00:00<00:25, 3356.76it/s]


  3%|▎         | 2699/89251 [00:00<00:26, 3281.12it/s]


  3%|▎         | 3059/89251 [00:00<00:25, 3363.79it/s]


  4%|▍         | 3404/89251 [00:01<00:25, 3372.03it/s]


  4%|▍         | 3744/89251 [00:01<00:25, 3373.02it/s]


  5%|▍         | 4076/89251 [00:01<00:27, 3149.84it/s]


  5%|▍         | 4446/89251 [00:01<00:25, 3290.48it/s]


  5%|▌         | 4802/89251 [00:01<00:25, 3359.99it/s]


  6%|▌         | 5139/89251 [00:01<00:25, 3296.72it/s]


  6%|▌         | 5470/89251 [00:01<00:25, 3283.71it/s]


  7%|▋         | 5820/89251 [00:01<00:24, 3

 56%|█████▌    | 49563/89251 [00:14<00:11, 3396.53it/s]


 56%|█████▌    | 49904/89251 [00:14<00:11, 3363.10it/s]


 56%|█████▋    | 50244/89251 [00:14<00:11, 3366.77it/s]


 57%|█████▋    | 50581/89251 [00:14<00:11, 3320.74it/s]


 57%|█████▋    | 50930/89251 [00:15<00:11, 3362.65it/s]


 57%|█████▋    | 51279/89251 [00:15<00:11, 3392.63it/s]


 58%|█████▊    | 51671/89251 [00:15<00:10, 3528.34it/s]


 58%|█████▊    | 52026/89251 [00:15<00:10, 3506.27it/s]


 59%|█████▊    | 52400/89251 [00:15<00:10, 3565.79it/s]


 59%|█████▉    | 52758/89251 [00:15<00:10, 3562.25it/s]


 60%|█████▉    | 53134/89251 [00:15<00:09, 3611.74it/s]


 60%|█████▉    | 53496/89251 [00:15<00:10, 3522.33it/s]


 60%|██████    | 53850/89251 [00:15<00:10, 3438.10it/s]


 61%|██████    | 54195/89251 [00:15<00:10, 3393.72it/s]


 61%|██████    | 54536/89251 [00:16<00:10, 3381.12it/s]


 61%|██████▏   | 54875/89251 [00:16<00:10, 3346.48it/s]


 62%|██████▏   | 55228/89251 [00:16<00:10, 3392.34it/s]


 62%|██████▏  

In [301]:
fastText_model_name = applyFastText(lemmatized_names, df_train.sold_fast)

In [304]:
train_ft_probas_pos_name = get_fastText_proba(fastText_model_name, lemmatized_names) 

In [305]:
np.savetxt('TRAIN_fastText_name_posproba.csv', train_ft_probas_pos_name, delimiter=',', encoding='utf-8')

#### Test values

In [306]:
test_ft_probas_pos_name = get_fastText_proba(fastText_model_name, test_lemmatized_names)

In [307]:
np.savetxt('TEST_fastText_name_posproba.csv', test_ft_probas_pos_name, delimiter=',', encoding='utf-8')

## Now replacing `desc_text` and `name_text` with probas

In [274]:
X = df_train.drop(['sold_fast', 'properties', 'product_id', 'owner_id', 'desc_text', 'name_text'], axis=1)
y = df_train['sold_fast']

In [310]:
desc_proba = pd.read_csv('TRAIN_fastText_desc_posproba.csv', header=None)
name_proba = pd.read_csv('TRAIN_fastText_name_posproba.csv', header=None)
X['desc_prob'] = desc_proba.values
X['name_prob'] = name_proba.values
X.head()

,category_id,city,date_created,delivery_available,img_num,lat,long,payment_available,price,product_type,region,sold_mode,subcategory_id,desc_prob,name_prob
1,4,Краснодар,2018-10-08,False,3,45.0686,38.9518,True,500.0,1,Краснодарский край,1,410,0.270319,0.313682
2,4,Тюмень,2018-06-18,False,2,57.1840,65.5674,False,300.0,1,Тюменская область,1,405,0.517638,0.442623
4,9,Омск,2018-07-31,True,1,54.9889,73.4312,True,1100.0,1,Омская область,1,908,0.106516,0.151561
6,3,Санкт-Петербург,2018-04-17,False,4,59.9590,30.4877,True,5000.0,1,Ленинградская область,1,312,0.173627,0.259099
10,5,Москва,2018-02-09,False,2,55.6473,37.4118,True,2000.0,1,Московская область,1,504,0.083777,0.398533


In [311]:
def cat2proba_dict(values, labels):
    cat_dict = {}
    probas = []
    for aval in np.unique(values):
        labels_for_val = labels[values == aval]
        n = len(labels_for_val)
        n_1 = len(labels_for_val[labels_for_val == 1])
        p = 1.0 * n_1 / n
        cat_dict[aval] = p
    return cat_dict

def cat2proba(cat_dict, values):
    probas = []
    for aval in values:
        if aval in cat_dict:
            probas.append(cat_dict[aval])
        else:
            probas.append(0)
    return np.array(probas)
def date2ymd(date):
    date_plitted = date.split('-')
    year = int(date_plitted[0])
    month = int(date_plitted[1])
    day = int(date_plitted[2])
    return [year, month, day]

In [312]:
category_id_dict = cat2proba_dict(df_train['category_id'].values, y)
subcategory_id_dict = cat2proba_dict(df_train['subcategory_id'].values, y)

city_dict = cat2proba_dict(df_train['city'].values, y)
region_dict = cat2proba_dict(df_train['region'].values, y)

In [313]:
def preprocess(X, category_id_dict, subcategory_id_dict, city_dict, region_dict):
    dates = np.array([date2ymd(date) for date in X.loc[:, 'date_created'].values])
    #year is the same everywhere, getting rid of it
    X.loc[:, 'month'] = dates[:, 1]
    X.loc[:, 'day'] = dates[:, 2]
    X = X.drop(['date_created'], axis=1)
    
    X.loc[:, 'city'] = cat2proba(city_dict, X['city'].values)
    X.loc[:, 'region'] = cat2proba(region_dict, X['region'].values)
    
    X.loc[:, 'category_id'] = cat2proba(category_id_dict, X['category_id'].values)
    X.loc[:, 'subcategory_id'] = cat2proba(category_id_dict, X['subcategory_id'].values)
    
    X.loc[:, 'delivery_available'] = X.loc[:, 'delivery_available'] * 1.
    X.loc[:, 'payment_available'] = X.loc[:, 'payment_available'] * 1.
    
    return X

In [314]:
X = preprocess(X, category_id_dict, subcategory_id_dict, city_dict, region_dict)

In [315]:
X.head()

,category_id,city,delivery_available,img_num,lat,long,payment_available,price,product_type,region,sold_mode,subcategory_id,desc_prob,name_prob,month,day
1,0.258829,0.227393,0.0,3,45.0686,38.9518,1.0,500.0,1,0.226545,1,0,0.270319,0.313682,10,8
2,0.258829,0.226342,0.0,2,57.1840,65.5674,0.0,300.0,1,0.227446,1,0,0.517638,0.442623,6,18
4,0.162397,0.235826,1.0,1,54.9889,73.4312,1.0,1100.0,1,0.234856,1,0,0.106516,0.151561,7,31
6,0.237816,0.229844,0.0,4,59.9590,30.4877,1.0,5000.0,1,0.231020,1,0,0.173627,0.259099,4,17
10,0.238401,0.230473,0.0,2,55.6473,37.4118,1.0,2000.0,1,0.227684,1,0,0.083777,0.398533,2,9


In [331]:
X.shape

(351281, 16)

In [338]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [339]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [340]:
# clf = RandomForestClassifier(n_estimators=500, n_jobs=3, class_weight='balanced', max_depth=4)
clf = XGBClassifier(n_estimators=200, learning_rate=0.02, max_depth=8, n_jobs=3, colsample_bytree=0.7, scale_pos_weight=1.)
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.7, gamma=0,
       learning_rate=0.02, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=3,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1.0, seed=None,
       silent=None, subsample=1, verbosity=1)

In [341]:
y_val_pred = clf.predict_proba(X_val)[:, 1]

auc_val = roc_auc_score(y_val, y_val_pred)
print("Test AUC: ", auc_val)

Test AUC:  0.7873901851817262


## Transform test likewise

In [335]:
X_test = df_test.drop(['properties', 'product_id', 'owner_id', 'desc_text', 'name_text'], axis=1)
desc_proba = pd.read_csv('TEST_fastText_desc_posproba.csv', header=None)
name_proba = pd.read_csv('TEST_fastText_name_posproba.csv', header=None)
X_test['desc_prob'] = desc_proba.values
X_test['name_prob'] = name_proba.values
X_test = preprocess(X_test, category_id_dict, subcategory_id_dict, city_dict, region_dict)
X_test.head()

,category_id,city,delivery_available,img_num,lat,long,payment_available,price,product_type,region,sold_mode,subcategory_id,desc_prob,name_prob,month,day
0,0.162397,0.229716,1.0,3,55.2639,61.3972,1.0,650.0,1,0.229135,1,0,0.268471,0.172749,10,22
3,0.169870,0.248002,1.0,2,53.3426,83.7380,1.0,750.0,1,0.242036,1,0,0.057047,0.125106,11,9
8,0.293339,0.250707,0.0,4,52.0883,113.4982,1.0,5500.0,1,0.250206,1,0,0.231807,0.184615,11,27
9,0.237816,0.230473,1.0,2,55.6932,37.5684,1.0,100.0,1,0.227684,1,0,0.123092,0.070957,11,25
19,0.169870,0.215649,1.0,2,54.7648,83.0884,1.0,700.0,1,0.232747,1,0,0.316553,0.212875,11,4


In [336]:
y_submit_pred = clf.predict_proba(X_test)[:, 1]

product_id = df_test['product_id'].values
data_submit = pd.DataFrame.from_dict({'product_id' : product_id, 'score' : y_submit_pred})
data_submit.to_csv('./to_submit.csv', sep = ',', index = False)

In [337]:
y_submit_pred.shape

(89251,)